In [2]:
import pandas as pd
import numpy as np
import ast
import re
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Configuración para mostrar más columnas y filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

def cargar_datos(archivos):
    """
    Carga los archivos CSV en DataFrames
    
    Args:
        archivos: Lista de rutas a los archivos CSV
    
    Returns:
        Diccionario con los DataFrames cargados
    """
    datos = {}
    for archivo in archivos:
        nombre = os.path.basename(archivo).split('.')[0]
        try:
            datos[nombre] = pd.read_csv(archivo)
            print(f"✅ Archivo {archivo} cargado correctamente")
        except Exception as e:
            print(f"❌ Error al cargar {archivo}: {e}")
    
    return datos

def convertir_listas(df):
    """
    Convierte las cadenas de texto que representan listas a listas reales de Python
    
    Args:
        df: DataFrame con las columnas a convertir
    
    Returns:
        DataFrame con las columnas convertidas
    """
    df_copy = df.copy()
    
    for col in df_copy.columns:
        try:
            # Intentar convertir cada valor de la columna a una lista Python
            df_copy[col] = df_copy[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        except (ValueError, SyntaxError):
            # Si falla, mantener la columna como está
            pass
    
    return df_copy

def extraer_dialogos(df):
    """
    Extrae los diálogos de la columna 'dialog' y crea nuevas columnas
    
    Args:
        df: DataFrame con la columna 'dialog'
    
    Returns:
        DataFrame con los diálogos extraídos
    """
    df_expanded = df.copy()
    
    # Extraer los diálogos individuales
    df_expanded['num_utterances'] = df_expanded['dialog'].apply(len)
    df_expanded['dialog_text'] = df_expanded['dialog'].apply(lambda x: ' '.join(x))
    
    # Extraer el primer y último diálogo (pueden ser útiles para análisis)
    df_expanded['first_utterance'] = df_expanded['dialog'].apply(lambda x: x[0] if len(x) > 0 else '')
    df_expanded['last_utterance'] = df_expanded['dialog'].apply(lambda x: x[-1] if len(x) > 0 else '')
    
    return df_expanded

def normalizar_actos_emociones(df):
    """
    Normaliza los actos y emociones, creando columnas numéricas
    
    Args:
        df: DataFrame con las columnas 'act' y 'emotion'
    
    Returns:
        DataFrame con actos y emociones normalizados y encoders utilizados
    """
    df_norm = df.copy()
    
    # Crear columnas para estadísticas de actos y emociones
    df_norm['act_counts'] = df_norm['act'].apply(lambda x: len(x))
    df_norm['emotion_counts'] = df_norm['emotion'].apply(lambda x: len(x))
    
    # Verificar que las longitudes coincidan
    df_norm['lengths_match'] = df_norm.apply(
        lambda row: len(row['dialog']) == len(row['act']) == len(row['emotion']), 
        axis=1
    )
    
    # Extraer actos y emociones más frecuentes
    def most_common(lst):
        if not lst:
            return None
        return max(set(lst), key=lst.count)
    
    df_norm['most_common_act'] = df_norm['act'].apply(most_common)
    df_norm['most_common_emotion'] = df_norm['emotion'].apply(most_common)
    
    # Codificar actos y emociones más frecuentes
    act_encoder = LabelEncoder()
    emotion_encoder = LabelEncoder()
    
    # Ajustar encoders con todos los valores posibles
    all_acts = [act for acts in df_norm['most_common_act'].dropna() for act in [acts]]
    all_emotions = [emotion for emotions in df_norm['most_common_emotion'].dropna() for emotion in [emotions]]
    
    act_encoder.fit(all_acts)
    emotion_encoder.fit(all_emotions)
    
    # Transformar valores
    df_norm['most_common_act_encoded'] = df_norm['most_common_act'].apply(
        lambda x: act_encoder.transform([x])[0] if x is not None else np.nan
    )
    
    df_norm['most_common_emotion_encoded'] = df_norm['most_common_emotion'].apply(
        lambda x: emotion_encoder.transform([x])[0] if x is not None else np.nan
    )
    
    # Crear mapeos para referencia
    act_mapping = {i: label for i, label in enumerate(act_encoder.classes_)}
    emotion_mapping = {i: label for i, label in enumerate(emotion_encoder.classes_)}
    
    return df_norm, {'act_encoder': act_encoder, 'emotion_encoder': emotion_encoder, 
                     'act_mapping': act_mapping, 'emotion_mapping': emotion_mapping}

def calcular_estadisticas_texto(df):
    """
    Calcula estadísticas sobre los textos de diálogo
    
    Args:
        df: DataFrame con la columna 'dialog_text'
    
    Returns:
        DataFrame con estadísticas de texto añadidas
    """
    df_stats = df.copy()
    
    # Calcular longitud de texto
    df_stats['dialog_length'] = df_stats['dialog_text'].apply(len)
    
    # Calcular número de palabras
    df_stats['word_count'] = df_stats['dialog_text'].apply(lambda x: len(x.split()))
    
    # Calcular longitud promedio de palabras
    df_stats['avg_word_length'] = df_stats['dialog_text'].apply(
        lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0
    )
    
    return df_stats

def visualizar_distribucion(df, encoders, conjunto):
    """
    Visualiza la distribución de actos y emociones
    
    Args:
        df: DataFrame normalizado
        encoders: Diccionario con encoders y mapeos
        conjunto: Nombre del conjunto de datos (train, test, validation)
    """
    plt.figure(figsize=(18, 8))
    
    # Distribución de actos
    plt.subplot(1, 2, 1)
    act_counts = df['most_common_act'].value_counts().head(10)
    sns.barplot(x=act_counts.index, y=act_counts.values)
    plt.title(f'Top 10 Actos más frecuentes - {conjunto}')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    
    # Distribución de emociones
    plt.subplot(1, 2, 2)
    emotion_counts = df['most_common_emotion'].value_counts().head(10)
    sns.barplot(x=emotion_counts.index, y=emotion_counts.values)
    plt.title(f'Top 10 Emociones más frecuentes - {conjunto}')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    
    plt.savefig(f'distribucion_{conjunto}.png')
    plt.close()
    
    # Estadísticas de longitud de diálogo
    plt.figure(figsize=(12, 6))
    sns.histplot(df['num_utterances'], bins=20, kde=True)
    plt.title(f'Distribución de número de expresiones por diálogo - {conjunto}')
    plt.xlabel('Número de expresiones')
    plt.ylabel('Frecuencia')
    plt.savefig(f'longitud_dialogos_{conjunto}.png')
    plt.close()
    
    print(f"✅ Visualizaciones guardadas para el conjunto {conjunto}")

def procesar_y_guardar(archivos_csv, directorio_salida='datos_normalizados'):
    """
    Procesa los archivos CSV y guarda los resultados en formato Parquet
    
    Args:
        archivos_csv: Lista de rutas a los archivos CSV
        directorio_salida: Directorio donde guardar los archivos Parquet
    """
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
        print(f"✅ Directorio creado: {directorio_salida}")
    
    # Cargar datos
    datos = cargar_datos(archivos_csv)
    
    # Procesar cada conjunto de datos
    dataframes_procesados = {}
    encoders = None
    
    for nombre, df in datos.items():
        print(f"\n{'='*80}")
        print(f"PROCESANDO CONJUNTO: {nombre}")
        print(f"{'='*80}")
        
        # Paso 1: Convertir strings a listas
        print("1️⃣ Convirtiendo strings a listas...")
        df_listas = convertir_listas(df)
        
        # Paso 2: Extraer y procesar diálogos
        print("2️⃣ Extrayendo información de diálogos...")
        df_dialogos = extraer_dialogos(df_listas)
        
        # Paso 3: Normalizar actos y emociones
        print("3️⃣ Normalizando actos y emociones...")
        if nombre == 'train':
            # Solo entrenar encoders en el conjunto de entrenamiento
            df_norm, encoders = normalizar_actos_emociones(df_dialogos)
        else:
            # Usar los encoders del conjunto de entrenamiento
            df_norm, _ = normalizar_actos_emociones(df_dialogos)
        
        # Paso 4: Calcular estadísticas de texto
        print("4️⃣ Calculando estadísticas de texto...")
        df_final = calcular_estadisticas_texto(df_norm)
        
        # Guardar DataFrame procesado
        dataframes_procesados[nombre] = df_final
        
        # Visualizar distribuciones
        print("5️⃣ Generando visualizaciones...")
        visualizar_distribucion(df_final, encoders, nombre)
        
        # Guardar en formato Parquet
        ruta_parquet = os.path.join(directorio_salida, f"{nombre}.parquet")
        df_final.to_parquet(ruta_parquet, index=False)
        print(f"✅ Datos guardados en: {ruta_parquet}")
        
        # Mostrar estadísticas básicas
        print("\n📊 ESTADÍSTICAS BÁSICAS:")
        print(f"- Número de diálogos: {len(df_final)}")
        print(f"- Longitud promedio de diálogo (caracteres): {df_final['dialog_length'].mean():.2f}")
        print(f"- Número promedio de palabras por diálogo: {df_final['word_count'].mean():.2f}")
        print(f"- Número promedio de expresiones por diálogo: {df_final['num_utterances'].mean():.2f}")
        
        # Verificar integridad
        print(f"\n🔍 VERIFICACIÓN DE INTEGRIDAD:")
        print(f"- Filas con longitudes inconsistentes: {(~df_final['lengths_match']).sum()} ({(~df_final['lengths_match']).sum() / len(df_final) * 100:.2f}%)")
    
    # Guardar mapeos de encoders
    if encoders:
        import json
        
        # Convertir mapeos a formato serializable
        act_mapping_serializable = {str(k): str(v) for k, v in encoders['act_mapping'].items()}
        emotion_mapping_serializable = {str(k): str(v) for k, v in encoders['emotion_mapping'].items()}
        
        with open(os.path.join(directorio_salida, 'act_mapping.json'), 'w') as f:
            json.dump(act_mapping_serializable, f)
        
        with open(os.path.join(directorio_salida, 'emotion_mapping.json'), 'w') as f:
            json.dump(emotion_mapping_serializable, f)
        
        print(f"✅ Mapeos de encoders guardados en el directorio {directorio_salida}")
    
    return dataframes_procesados

# Ejecutar el procesamiento
archivos = ['train.csv', 'test.csv', 'validation.csv']
dataframes = procesar_y_guardar(archivos)

print("\n✅ Procesamiento CRISP-DM completado. Los datos están listos para modelado.")

# Mostrar un resumen final
print("\n📋 RESUMEN FINAL:")
for nombre, df in dataframes.items():
    print(f"- {nombre}: {df.shape[0]} filas x {df.shape[1]} columnas")

# Verificar columnas en los conjuntos procesados
train_cols = set(dataframes['train'].columns)
test_cols = set(dataframes['test'].columns)
val_cols = set(dataframes['validation'].columns)

print(f"\n✅ Todas las columnas son iguales en los conjuntos procesados: {train_cols == test_cols == val_cols}")


✅ Directorio creado: datos_normalizados
✅ Archivo train.csv cargado correctamente
✅ Archivo test.csv cargado correctamente
✅ Archivo validation.csv cargado correctamente

PROCESANDO CONJUNTO: train
1️⃣ Convirtiendo strings a listas...
2️⃣ Extrayendo información de diálogos...
3️⃣ Normalizando actos y emociones...
4️⃣ Calculando estadísticas de texto...
5️⃣ Generando visualizaciones...
✅ Visualizaciones guardadas para el conjunto train
✅ Datos guardados en: datos_normalizados\train.parquet

📊 ESTADÍSTICAS BÁSICAS:
- Número de diálogos: 11118
- Longitud promedio de diálogo (caracteres): 485.26
- Número promedio de palabras por diálogo: 106.68
- Número promedio de expresiones por diálogo: 1.00

🔍 VERIFICACIÓN DE INTEGRIDAD:
- Filas con longitudes inconsistentes: 11118 (100.00%)

PROCESANDO CONJUNTO: test
1️⃣ Convirtiendo strings a listas...
2️⃣ Extrayendo información de diálogos...
3️⃣ Normalizando actos y emociones...
4️⃣ Calculando estadísticas de texto...
5️⃣ Generando visualizaciones.